## BadフィードバックがついているデータにはID:0を文章に追加してコサイン類似検索する
結論、ID:0という文言をつけることで、BADフィードバックついた質問のsimilarityを少し下げることが出来た。

In [31]:
'''
    プロトタイピング用のパスと、Botライブラリーパスを取得／設定します
'''
import sys
import os

prototype_dir = os.path.join(os.getcwd(), '..')
prototype_dir = os.path.abspath(prototype_dir)

learning_dir = os.path.join(prototype_dir, '..')
learning_dir = os.path.abspath(learning_dir)
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

print('prototype_dir=%s\nlearning_dir=%s' % (prototype_dir, learning_dir))

prototype_dir=/Users/harada
learning_dir=/Users


In [32]:
import pandas as pd
from IPython.display import display, HTML

feedbacks =  pd.DataFrame({
    'id': [0, 1, 2],  # BadフィードバックはID:0を設定する
    'question': [
        'ピアノの先生を見つけたい',
        'ダミーのデータ1',
        'ダミーのデータ2',
    ],
})
display(feedbacks)

,id,question
0,0,ピアノの先生を見つけたい
1,1,ダミーのデータ1
2,2,ダミーのデータ2


In [33]:
question_answers =  pd.DataFrame({
    'id': [11,12,13],
    'question': [
        'ピアノ持ってない',
        'うちの子供は2歳です。ピアノを始めるにはまだ早いでしょうか？ ',
        '国立音大の先生を探しているのですが。',
    ],
    'answer': [
        'お申込時点でピアノをご家庭にお持ちでなくても、基本は問題ありません。',
        '一般に、ピアノを習うのは早ければ早いほどいい、と言われます。',
        '音大勤務の指定については、お申込時にご要望欄へ記入ください。個別にご相談に応じます。',
    ]
})
display(question_answers)

,answer,id,question
0,お申込時点でピアノをご家庭にお持ちでなくても、基本は問題ありません。,11,ピアノ持ってない
1,一般に、ピアノを習うのは早ければ早いほどいい、と言われます。,12,うちの子供は2歳です。ピアノを始めるにはまだ早いでしょうか？
2,音大勤務の指定については、お申込時にご要望欄へ記入ください。個別にご相談に応じます。,13,国立音大の先生を探しているのですが。


In [34]:
'''
普通のコサイン類似検索をする
'''
from app.core.tokenizer.mecab_tokenizer import MecabTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer as SkTfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tokenized_sentences = MecabTokenizer().tokenize(question_answers['question'])
vectorizer = SkTfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b')
vectorized_features = vectorizer.fit_transform(tokenized_sentences)

print(vectorized_features.toarray().shape)


X = ['ピアノの先生を見つけたいです']
tokenized_questions = MecabTokenizer().tokenize(X)
vectorized_questions = vectorizer.transform(tokenized_questions)

print(vectorized_questions.toarray().shape)

similarities = cosine_similarity(vectorized_features, vectorized_questions)

# question_answersのid:13のデータが類似度が高いという結果が出る
print(similarities)

(3, 13)
(1, 13)
[[ 0.28671097]
 [ 0.16723635]
 [ 0.45954803]]


In [35]:
'''
feedbacksからコサイン類似検索してidを取得する
'''
questions = feedbacks['question']
tokenized_sentences = MecabTokenizer().tokenize(questions)
vectorized_features = vectorizer.transform(tokenized_sentences)
print(vectorized_features.toarray().shape)

X = ['ピアノの先生を見つけたいです']
tokenized_questions = MecabTokenizer().tokenize(X)
vectorized_questions = vectorizer.transform(tokenized_questions)
print(vectorized_questions.toarray().shape)

similarities = cosine_similarity(vectorized_features, vectorized_questions)
similarities = similarities.flatten()
feedbacks['similarity'] = similarities

# 最もsimilarityが高いidを取得する
display(feedbacks)
feedbacks = feedbacks.sort_values(by='similarity', ascending=False)
display(feedbacks[:1])
predicted_id = feedbacks[:1]['id'][0]
print(predicted_id)

(3, 13)
(1, 13)


,id,question,similarity
0,0,ピアノの先生を見つけたい,1.0
1,1,ダミーのデータ1,0.0
2,2,ダミーのデータ2,0.0


,id,question,similarity
0,0,ピアノの先生を見つけたい,1.0


0


In [36]:
'''
idを加えて普通のコサイン類似検索をする
'''
from app.core.tokenizer.mecab_tokenizer import MecabTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer as SkTfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

questions = question_answers['question']
questions = question_answers[['question', 'id']].apply(lambda x: '{} {}'.format(x[0], x[1]), axis=1)
display(questions)

tokenized_sentences = MecabTokenizer().tokenize(questions)
print(tokenized_sentences)
vectorized_features = vectorizer.fit_transform(tokenized_sentences)
print(vectorized_features.toarray().shape)

X = ['ピアノの先生を見つけたいです']
X_new = [X[0] + ' ' + str(predicted_id)]
print(X_new)
tokenized_questions = MecabTokenizer().tokenize(X_new)
print(tokenized_questions)
vectorized_questions = vectorizer.transform(tokenized_questions)
print(vectorized_questions.toarray().shape)

# idを加えてコサイン類似検索することで、正解データのｓimilarityが  0.45954803 => 0.39798027 となり、わずかに下がった。
# 十分ではないがある程度想定していた方向の挙動となった
similarities = cosine_similarity(vectorized_features, vectorized_questions)
display(similarities)

0                           ピアノ持ってない 11
1    うちの子供は2歳です。ピアノを始めるにはまだ早いでしょうか？  12
2                 国立音大の先生を探しているのですが。 13
dtype: object

['ピアノ 持つ ない １１', 'うち 子供 ２ 歳 ピアノ 始める まだ 早い １２', '国立音大 先生 探す １３']
(3, 16)
['ピアノの先生を見つけたいです 0']
['ピアノ 先生 見つける ０']
(1, 16)


array([[ 0.24337446],
       [ 0.15718688],
       [ 0.39798027]])